In [1]:
using BenchmarkTools
using Revise

In [2]:
using CSV
using DataFrames
using JSON

using DataIO
using GCTGMT
using Normalization
using Support

using FeatureSetEnrichment

┌ Info: Precompiling FeatureSetEnrichment [d7bef1b9-4643-4ed1-9ceb-ed43dd382ec0]
└ @ Base loading.jl:1317


HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-994060335855289353\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-1617827721267030514\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-994060335855289353\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

In [3]:
name_to_set_to_element_ = Dict(
    splitpath(path)[end] => read_gmt(path) for
    path in read_directory("/Users/kwat/Downloads/gene_sets/")
)

;

In [4]:
method_ = Vector{String}()

for method in keys(score_set_new(["a", "b"], [-1.0, 1.0], ["a"]; plot = false))

    push!(method_, string(method, " extreme"), string(method, " area"))

end

n_method = length(method_)

;

The minimum and the maximum have the same absolute value; returning the minimum.
The minimum and the maximum have the same absolute value; returning the minimum.
The minimum and the maximum have the same absolute value; returning the minimum.
The minimum and the maximum have the same absolute value; returning the minimum.
The minimum and the maximum have the same absolute value; returning the minimum.
The minimum and the maximum have the same absolute value; returning the minimum.
The minimum and the maximum have the same absolute value; returning the minimum.
The minimum and the maximum have the same absolute value; returning the minimum.
The minimum and the maximum have the same absolute value; returning the minimum.
The minimum and the maximum have the same absolute value; returning the minimum.
The minimum and the maximum have the same absolute value; returning the minimum.
The minimum and the maximum have the same absolute value; returning the minimum.


In [5]:
benchmarks_directory_path = "/Users/kwat/Downloads/benchmarks/"

"/Users/kwat/Downloads/benchmarks/"

In [6]:
benchmark_ = Vector{String}()

benchmark_size_ = Vector{Int64}()

benchmark_x_method_rows = Vector{Vector{Float64}}()

;

In [7]:
for benchmark_directory_path in read_directory(benchmarks_directory_path)

    #
    name = splitpath(benchmark_directory_path)[end]

    println(name)

    #
    element_, score_ =
        eachcol(read_data(joinpath(benchmark_directory_path, "gene_x_score.tsv")))

    #
    json_dict = JSON.parse(open(joinpath(benchmark_directory_path, "gene_sets.json")))

    #
    set_to_element_ = Dict{String,Vector{String}}()

    for gmt_path in json_dict["gene_sets_tested"]

        merge!(set_to_element_, name_to_set_to_element_[splitpath(gmt_path)[end]])

    end

    set_ = sort(collect(keys(set_to_element_)))

    n_set = length(set_)

    println("    ", n_set, " set.")

    #
    set_to_method_to_result = sort(score_set_new(element_, score_, set_to_element_))

    @assert all(set_ .== collect(keys(set_to_method_to_result)))

    set_x_method = Matrix{Float64}(undef, n_set, n_method)

    for (set_i, (set, method_to_result)) in enumerate(set_to_method_to_result)

        @assert set_[set_i] == set

        set_x_method_row = []

        for result in values(method_to_result)

            append!(set_x_method_row, result[2:3])

        end

        set_x_method[set_i, :] = set_x_method_row

    end

    #
    for set in json_dict["gene_sets_positive"]

        #
        print("    ")

        if !in(set, set_)

            println(set, " (missing)")

            continue

        end

        benchmark = string(name, '.', set)

        benchmark_size = length(set_to_element_[set])

        println(set, " (", benchmark_size, ")")

        #
        push!(benchmark_, benchmark)

        push!(benchmark_size_, benchmark_size)

        #
        benchmark_x_method_row = Vector{Float64}()

        for (method_i, set_score_) in enumerate(eachcol(set_x_method))

            #
            is_negative_ = set_score_ .< 0

            is_positive_ = 0 .<= set_score_

            #
            set_negative_ = set_[is_negative_]

            set_positive_ = set_[is_positive_]

            #
            negative_ = .-normalize(.-set_score_[is_negative_], "1224") / sum(is_negative_)

            positive_ = normalize(set_score_[is_positive_], "1224") / sum(is_positive_)

            #
            i_negative = findfirst(set_negative_ .== set)

            i_positive = findfirst(set_positive_ .== set)

            #
            if i_negative != nothing

                benchmark_score = negative_[i_negative]

            elseif i_positive != nothing

                benchmark_score = positive_[i_positive]

            else

                benchmark_score = set_score_[findfirst(set_ .== set)]

                println("        ", method_[method_i], " returned ", benchmark_score)

            end

            push!(benchmark_x_method_row, benchmark_score)

        end

        #
        push!(benchmark_x_method_rows, benchmark_x_method_row)

    end

end

1
    4951 set.
    AIGNER_ZEB1_TARGETS (35)
    SANSOM_APC_TARGETS_UP (126)
    BCAT_GDS748_UP (48)
    BCAT.100_UP.V1_UP (49)
    PID_WNT_SIGNALING_PATHWAY (28)
    LIU_CDX2_TARGETS_UP (36)
    LUCAS_HNF4A_TARGETS_UP (58)
    BCAT_GDS748 (missing)
10
    4762 set.
    LANG_MYB_FAMILY_TARGETS (29)
    PID_CMYB_PATHWAY (84)
100
    10679 set.
    GO_CELL_CYCLE_G1_S_PHASE_TRANSITION (111)
    REACTOME_M_G1_TRANSITION (81)
    REACTOME_MITOTIC_M_M_G1_PHASES (172)
    KEGG_DNA_REPLICATION (36)
    REACTOME_DNA_REPLICATION (192)
    EGUCHI_CELL_CYCLE_RB1_TARGETS (23)
    WHITFIELD_CELL_CYCLE_LITERATURE (44)
101
    4762 set.
    BIOCARTA_NFKB_PATHWAY (23)
    TIAN_TNF_SIGNALING_VIA_NFKB (28)
    BIOCARTA_TNFR2_PATHWAY (18)
    SANA_TNF_SIGNALING_UP (83)
    PHONG_TNF_TARGETS_UP (63)
    SCHOEN_NFKB_SIGNALING (34)
    GILMORE_CORE_NFKB_PATHWAY (14)
102
    4762 set.
    REACTOME_ALPHA_LINOLENIC_ACID_ALA_METABOLISM (12)
    KEGG_BETA_ALANINE_METABOLISM (22)
103
    4762 set.
    REACTOME_CEL

In [8]:
#
benchmark_x_method = convert_vector_of_vector_to_matrix(benchmark_x_method_rows)

benchmark_x_method = DataFrame(benchmark_x_method, method_)

insertcols!(benchmark_x_method, 1, "Benchmark" => benchmark_)

#
benchmark_x_method_path = "benchmark_x_method.tsv"

CSV.write(benchmark_x_method_path, benchmark_x_method)

benchmark_x_method = read_data(benchmark_x_method_path)

,Benchmark,Is KS < extreme
,String,Float64
1,1.AIGNER_ZEB1_TARGETS,0.995307
2,1.SANSOM_APC_TARGETS_UP,0.856229
3,1.BCAT_GDS748_UP,0.912969
4,1.BCAT.100_UP.V1_UP,0.956058
5,1.PID_WNT_SIGNALING_PATHWAY,0.393771
6,1.LIU_CDX2_TARGETS_UP,0.938993
7,1.LUCAS_HNF4A_TARGETS_UP,0.957338
8,10.LANG_MYB_FAMILY_TARGETS,0.501956
9,10.PID_CMYB_PATHWAY,0.717719


In [9]:
#
benchmark_x_method_metric = Matrix{Int64}(undef, length(benchmark_), n_method)

for (benchmark_i, benchmark_score_) in
    enumerate(eachrow(select(benchmark_x_method, Not("Benchmark"))))

    #
    classic_score = benchmark_score_["Is KS < extreme"]

    #
    benchmark_score_ = Vector(benchmark_score_)

    #
    if classic_score < 0

        metric_ = benchmark_score_ .< classic_score

    else

        metric_ = classic_score .< benchmark_score_

    end

    #
    benchmark_x_method_metric[benchmark_i, :] = metric_

end

In [10]:
function sum_sort(m::Matrix)::Nothing

    for (method, n_better) in zip(sort_like(sum(eachrow(m)), method_)...)

        println(method, "    ", n_better)

    end

    return nothing

end

sum_sort (generic function with 1 method)

In [12]:
buffer = "="^8

benchmark_size_maximum = maximum(benchmark_size_)

println(buffer, " < ", benchmark_size_maximum, " ", buffer)

sum_sort(benchmark_x_method_metric)

increment = 50

for (minimum_size, maximum_size) in [
    ((i - 1) * increment, i * increment) for
    i = 1:Int64(ceil(benchmark_size_maximum / increment))
]

    #
    println(buffer, " ", minimum_size, " < ", maximum_size, " ", buffer)

    #
    is_selected = minimum_size .< benchmark_size_ .< maximum_size
    
    n_selected = sum(is_selected)
    
    println(n_selected)

    if 1 < n_selected

        sum_sort(benchmark_x_method_metric[is_selected, :])

    end

end

======== < 1193 ========
0    Is KS < extreme
0    Is KS > extreme
0    Is JSD2 > extreme
1    Is KS > area
1    Is KS <> extreme
1    Is JSD2 > area
1    A KS > area
1    A JSD1w > area
1    A JSD2 > area
1    A JSD2w > area
2    Is JSD4 > area
3    A KS > extreme
3    A JSD2 > extreme
3    A JSD4 > area
4    A JSD1w > extreme
4    A JSD2w > extreme
5    Is JSD4 > extreme
8    A JSD4 > extreme
31    A JSD3 <> area
38    A JSD3 <> extreme
41    A JSD3 < area
51    Is JSD3 <> extreme
58    A JSD1 <> area
67    A JSD3 < extreme
69    A JSD4 <> extreme
73    Is JSD3 <> area
83    Is JSD3 < extreme
86    Is JSD3 < area
86    Is JSD4 <> extreme
100    A JSD1 <> extreme
106    A JSD1 > extreme
111    Is JSD1 > extreme
117    Is JSD3 > extreme
121    A JSD3 > extreme
131    Is JSD1 <> extreme
139    Is JSD3 > area
151    A JSD3 > area
153    Is JSD1 <> area
153    A JSD2 < extreme
165    A JSD4 < extreme
173    A JSD4 <> area
175    A JSD1 < extreme
187    Is JSD4 < extreme
198    A KS < extr